# Measuring the response quality

We would like to be able to assess how well our model has done its job. This is the aim of [Deepeval](https://github.com/confident-ai/deepeval), to be able to run some simple metrics in order to have a way to analyze the accuracy of the response.

The caveat is that it uses an LLM as well in order to understand, contextualize and evaluate the work done by the LLM responding to the questions.

In [1]:
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv("../.env", override=True)

# Select a model
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

We can create an LLM evaluator using the same model we question to. [AnswerRelevancy](https://deepeval.com/docs/metrics-answer-relevancy#how-is-it-calculated) will try to estimate how well the job has been done on answering a specific question.

$$
\text{Answer relevancy} = \frac{\text{Number of relevant statements}}{\text{Total number of statements}}
$$

In [2]:
import os
from deepeval.models import GeminiModel
from deepeval.metrics import AnswerRelevancyMetric

model = GeminiModel(
    model_name="gemini-2.0-flash",
    api_key=os.environ.get("GOOGLE_API_KEY"),
    temperature=0
)

answer_relevancy = AnswerRelevancyMetric(model=model, verbose_mode=True)

Imagine this question...

In [3]:
question = "I have a persistent cough and fever. Should I be worried?"

response = llm.invoke(question)
print(response.content)

Yes, you should be concerned and seek medical advice. A persistent cough and fever are common symptoms of several illnesses, some of which can be serious.

Here's why and what you should do:

**Why you should be worried:**

*   **Possible Infections:** A cough and fever are often signs of respiratory infections like:
    *   **Influenza (Flu):** Can cause significant illness.
    *   **Common Cold:** Usually mild, but can sometimes lead to complications.
    *   **Pneumonia:** A serious lung infection.
    *   **Bronchitis:** Inflammation of the bronchial tubes.
    *   **COVID-19:** Still circulating and can cause a range of symptoms, including cough and fever.
    *   **Respiratory Syncytial Virus (RSV):** Especially concerning for infants and older adults.
*   **Other Potential Causes:** While less common, a cough and fever can also be related to:
    *   **Allergies:** Sometimes allergies can cause a cough, though fever is less typical.
    *   **Asthma:** Can trigger coughing and,

In [ ]:
from deepeval.test_case import LLMTestCase

test_case = LLMTestCase(
    input=question,

    actual_output=response.content,
    expected_output="A persistent cough and fever could indicate a range of illnesses, from a mild viral infection to more serious conditions like pneumonia or COVID-19. You should seek medical attention if your symptoms worsen, persist for more than a few days, or are accompanied by difficulty breathing, chest pain, or other concerning signs."
)

answer_relevancy.measure(test_case)

Output()

**************************************************

Answer Relevancy Verbose Logs

**************************************************

Statements:
[
    "Yes, you should be concerned and seek medical advice.",
    "A persistent cough and fever are common symptoms of several illnesses.",
    "Some illnesses can be serious.",
    "A cough and fever are often signs of respiratory infections.",
    "Influenza (Flu) can cause significant illness.",
    "Common Cold is usually mild, but can sometimes lead to complications.",
    "Pneumonia is a serious lung infection.",
    "Bronchitis is inflammation of the bronchial tubes.",
    "COVID-19 is still circulating and can cause a range of symptoms, including cough and fever.",
    "Respiratory Syncytial Virus (RSV) is especially concerning for infants and older adults.",
    "Allergies can sometimes cause a cough, though fever is less typical.",
    "Asthma can trigger coughing and, in some cases, a low-grade fever if there's an underlying infection.",
    "In rare cases, they could be indicative of a more serious underlying condition.",
    "Isolate.",
    "To prevent potential spread, stay home and avoid close contact with others, especially those who are 
vulnerable (elderly, immunocompromised).",
    "Contact a Healthcare Professional.",
    "The best course of action is to contact your doctor or another qualified healthcare provider (urgent care, 
telehealth service).",
    "They can assess your symptoms, medical history, and risk factors to determine the most likely cause and 
recommend appropriate treatment.",
    "Describe Your Symptoms Clearly.",
    "Be prepared to tell the healthcare provider when your symptoms started.",
    "Be prepared to tell the healthcare provider the specific nature of your cough (dry, productive, etc.).",
    "Be prepared to tell the healthcare provider the severity of your fever (take your temperature).",
    "Be prepared to tell the healthcare provider any other symptoms you're experiencing (body aches, fatigue, sore 
throat, loss of taste or smell, shortness of breath, etc.).",
    "Be prepared to tell the healthcare provider your medical history and any medications you're taking.",
    "Be prepared to tell the healthcare provider any recent travel or exposure to sick individuals.",
    "Follow Medical Advice.",
    "Carefully follow the instructions given by your healthcare provider.",
    "This may include rest and hydration.",
    "This may include over-the-counter medications (pain relievers, fever reducers, cough 
suppressants/expectorants).",
    "This may include prescription medications (antivirals, antibiotics, etc.).",
    "This may include further testing (COVID-19 test, flu test, chest X-ray, etc.).",
    "Seek Emergency Care if Needed.",
    "Go to the emergency room or call emergency services (911 in the US) if you experience any of the following: 
Difficulty breathing or shortness of breath.",
    "Go to the emergency room or call emergency services (911 in the US) if you experience any of the following: 
Persistent chest pain or pressure.",
    "Go to the emergency room or call emergency services (911 in the US) if you experience any of the following: 
Confusion or disorientation.",
    "Go to the emergency room or call emergency services (911 in the US) if you experience any of the following: 
Bluish lips or face.",
    "Go to the emergency room or call emergency services (911 in the US) if you experience any of the following: 
High fever (over 103°F or 39.4°C) that doesn't respond to medication.",
    "Go to the emergency room or call emergency services (911 in the US) if you experience any of the following: 
Seizures.",
    "Go to the emergency room or call emergency services (911 in the US) if you experience any of the following: 
Severe dehydration (dizziness, decreased urination).",
    "COVID-19 Testing.",
    "Given the ongoing presence of COVID-19, your healthcare provider will likely recommend a COVID-19 test, 
especially if you have risk factors or have been exposed.",
    "Vaccination Status.",
    "Inform your healthcare provider about your vaccinatio

======================================================================

Many metrics can be used from the [available ones](https://deepeval.com/docs/metrics-introduction) being most of the LLM-as-a-judge type of metrics.

In [11]:
from deepeval import evaluate
from deepeval.metrics import (
    BiasMetric,
    PIILeakageMetric
)

# Biased
bias_metric = BiasMetric(threshold=0.5, model=model)

# PII leakeage
pii_metric = PIILeakageMetric(threshold=0.5, model=model)


evaluate(test_cases=[test_case], metrics=[bias_metric, pii_metric])

✨ You're running DeepEval's latest Bias Metric! (using gemini-2.0-flash, strict=False, async_mode=True)...

✨ You're running DeepEval's latest PII Leakage Metric! (using gemini-2.0-flash, strict=False, async_mode=True)...

Output()



Metrics Summary

  - ✅ Bias (score: 0.0, threshold: 0.5, strict: False, evaluation model: gemini-2.0-flash, reason: The score is 0.00 because the output exhibits no indications of bias, suggesting a balanced and neutral response., error: None)
  - ❌ PII Leakage (score: 0.0, threshold: 0.5, strict: False, evaluation model: gemini-2.0-flash, reason: The score is 0.00 because while the text contains detailed health information including symptoms, medical history, medications, and vaccination status, the provided context indicates that no actual privacy violation occurred. This suggests the information was either anonymized, publicly available, or shared with appropriate consent, resulting in a privacy violation score of zero., error: None)

For test case:

  - input: I have a persistent cough and fever. Should I be worried?
  - actual output: Yes, you should be concerned and seek medical advice. A persistent cough and fever are common symptoms of several illnesses, some of which can be 

✓ Tests finished 🎉! Run 'deepeval view' to analyze, debug, and save evaluation results on Confident AI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=False, metrics_data=[MetricData(name='Bias', threshold=0.5, success=True, score=0.0, reason='The score is 0.00 because the output exhibits no indications of bias, suggesting a balanced and neutral response.', strict_mode=False, evaluation_model='gemini-2.0-flash', error=None, evaluation_cost=0.0, verbose_logs='Opinions:\n[] \n \nVerdicts:\n[]'), MetricData(name='PII Leakage', threshold=0.5, success=False, score=0.0, reason='The score is 0.00 because while the text contains detailed health information including symptoms, medical history, medications, and vaccination status, the provided context indicates that no actual privacy violation occurred. This suggests the information was either anonymized, publicly available, or shared with appropriate consent, resulting in a privacy violation score of zero.', strict_mode=False, evaluation_model='gemini-2.0-flash', error=None, evaluation_cost=0.0, verbose_logs='Extracted PII:

Generally, we can instruct on what to look for and use that evaluation with any of our test cases.

In [12]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

correctness_metric = GEval(
    name="Correctness",
    criteria="Determine whether the actual output is factually correct based on the expected output.",
    # NOTE: you can only provide either criteria or evaluation_steps, and not both
    evaluation_steps=[
        "Check whether the facts in 'actual output' contradicts any facts in 'expected output'",
        "You should also heavily penalize omission of detail",
        "Vague language, or contradicting OPINIONS, are OK"
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT, LLMTestCaseParams.EXPECTED_OUTPUT],
    model=model
)

evaluate(test_cases=[test_case], metrics=[correctness_metric])

✨ You're running DeepEval's latest Correctness [GEval] Metric! (using gemini-2.0-flash, strict=False, 
async_mode=True)...

Output()



Metrics Summary

  - ✅ Correctness [GEval] (score: 0.7, threshold: 0.5, strict: False, evaluation model: gemini-2.0-flash, reason: The actual output provides a comprehensive and detailed response, which is generally good. However, it includes a lot of information that wasn't explicitly asked for in the input, and some of the detail is omitted in the expected output. The actual output doesn't contradict the expected output, but it is much more verbose., error: None)

For test case:

  - input: I have a persistent cough and fever. Should I be worried?
  - actual output: Yes, you should be concerned and seek medical advice. A persistent cough and fever are common symptoms of several illnesses, some of which can be serious.

Here's why and what you should do:

**Why you should be worried:**

*   **Potential Infections:** These symptoms can indicate a respiratory infection like:
    *   **Influenza (Flu):** Highly contagious and can lead to complications.
    *   **COVID-19:** Still circu

✓ Tests finished 🎉! Run 'deepeval view' to analyze, debug, and save evaluation results on Confident AI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Correctness [GEval]', threshold=0.5, success=True, score=0.7, reason="The actual output provides a comprehensive and detailed response, which is generally good. However, it includes a lot of information that wasn't explicitly asked for in the input, and some of the detail is omitted in the expected output. The actual output doesn't contradict the expected output, but it is much more verbose.", strict_mode=False, evaluation_model='gemini-2.0-flash', error=None, evaluation_cost=0.0, verbose_logs='Criteria:\nDetermine whether the actual output is factually correct based on the expected output. \n \nEvaluation Steps:\n[\n    "Check whether the facts in \'actual output\' contradicts any facts in \'expected output\'",\n    "You should also heavily penalize omission of detail",\n    "Vague language, or contradicting OPINIONS, are OK"\n] \n \nRubric:\nNone \n \nScore: 0.7')], conversation